# 1. Importing modules and functions

In [1]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from molvs import standardize_smiles
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import joblib
import pickle
from numpy import savetxt
from padelpy import from_sdf
import shap
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
import glob
xml_files = glob.glob("fingerprints_xml/*.xml")
xml_files.sort()
xml_files

['fingerprints_xml\\AtomPairs2DFingerprintCount.xml',
 'fingerprints_xml\\AtomPairs2DFingerprinter.xml',
 'fingerprints_xml\\EStateFingerprinter.xml',
 'fingerprints_xml\\ExtendedFingerprinter.xml',
 'fingerprints_xml\\Fingerprinter.xml',
 'fingerprints_xml\\GraphOnlyFingerprinter.xml',
 'fingerprints_xml\\KlekotaRothFingerprintCount.xml',
 'fingerprints_xml\\KlekotaRothFingerprinter.xml',
 'fingerprints_xml\\MACCSFingerprinter.xml',
 'fingerprints_xml\\PubchemFingerprinter.xml',
 'fingerprints_xml\\SubstructureFingerprintCount.xml',
 'fingerprints_xml\\SubstructureFingerprinter.xml']

In [3]:
FP_list = ['AtomPairs2DCount',
 'AtomPairs2D',
 'EState',
 'CDKextended',
 'CDK',
 'CDKgraphonly',
 'KlekotaRothCount',
 'KlekotaRoth',
 'MACCS',
 'PubChem',
 'SubstructureCount',
 'Substructure']

In [4]:
fp = dict(zip(FP_list, xml_files))
fp

{'AtomPairs2DCount': 'fingerprints_xml\\AtomPairs2DFingerprintCount.xml',
 'AtomPairs2D': 'fingerprints_xml\\AtomPairs2DFingerprinter.xml',
 'EState': 'fingerprints_xml\\EStateFingerprinter.xml',
 'CDKextended': 'fingerprints_xml\\ExtendedFingerprinter.xml',
 'CDK': 'fingerprints_xml\\Fingerprinter.xml',
 'CDKgraphonly': 'fingerprints_xml\\GraphOnlyFingerprinter.xml',
 'KlekotaRothCount': 'fingerprints_xml\\KlekotaRothFingerprintCount.xml',
 'KlekotaRoth': 'fingerprints_xml\\KlekotaRothFingerprinter.xml',
 'MACCS': 'fingerprints_xml\\MACCSFingerprinter.xml',
 'PubChem': 'fingerprints_xml\\PubchemFingerprinter.xml',
 'SubstructureCount': 'fingerprints_xml\\SubstructureFingerprintCount.xml',
 'Substructure': 'fingerprints_xml\\SubstructureFingerprinter.xml'}

# 2.Data entry and curation work set

In [5]:
uploaded_file_ws="datasets/HDAC3_work.sdf"
supplier_ws = Chem.ForwardSDMolSupplier(uploaded_file_ws,sanitize=False)
failed_mols_ws = []
all_mols_ws =[]
wrong_structure_ws=[]
wrong_smiles_ws=[]
y_tr = []
y_bad_index=[]

for i, m in enumerate(supplier_ws):
    structure = Chem.Mol(m)
    all_mols_ws.append(structure)
    y_tr.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ws.append(m)
        wrong_smiles_ws.append(Chem.MolToSmiles(m))
        wrong_structure_ws.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ws), 'molecules')
print('Failed data: ', len(failed_mols_ws), 'molecules')
number_ws =[]
for i in range(len(failed_mols_ws)):
        number_ws.append(str(i+1))
bad_molecules_ws = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ws, 'SMILES of wrong structure: ': wrong_smiles_ws, 'No.': number_ws}, index=None)
bad_molecules_ws = bad_molecules_ws.set_index('No.')
bad_molecules_ws

Original data:  1400 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [6]:
y_tr[:] = [x for i,x in enumerate(y_tr) if i not in y_bad_index]

In [7]:
len(y_tr)

1400

# 3.Standardization SDF file for work set

In [8]:
all_mols_ws[:] = [x for i,x in enumerate(all_mols_ws) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ws)):
    record = Chem.MolToSmiles(all_mols_ws[i])
    records.append(record)

moldf_ws = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ws.append(m)
    
print('Kept data: ', len(moldf_ws), 'molecules')

Kept data:  1400 molecules


In [9]:
records_ws = []
for i in range(len(moldf_ws)):
    record = Chem.MolToSmiles(moldf_ws[i])
    records_ws.append(record)

In [10]:
df = pd.DataFrame(records_ws, columns=["Smiles"])
df.to_csv('datasets/molecule_ws.smi', sep=',', index=False, header=False)

# 4.Data entry and curation test set

In [11]:
uploaded_file_ts="datasets/HDAC3_test.sdf"
supplier_ts = Chem.ForwardSDMolSupplier(uploaded_file_ts,sanitize=False)
failed_mols_ts = []
all_mols_ts =[]
wrong_structure_ts=[]
wrong_smiles_ts=[]
y_ts = []
y_bad_index=[]
for i, m in enumerate(supplier_ts):
    structure = Chem.Mol(m)
    all_mols_ts.append(structure)
    y_ts.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ts.append(m)
        wrong_smiles_ts.append(Chem.MolToSmiles(m))
        wrong_structure_ts.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ts), 'molecules')
print('Failed data: ', len(failed_mols_ts), 'molecules')
number_ts =[]
for i in range(len(failed_mols_ts)):
        number_ts.append(str(i+1))
bad_molecules_ts = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ts, 'SMILES of wrong structure: ': wrong_smiles_ts, 'No.': number_ts}, index=None)
bad_molecules_ts = bad_molecules_ts.set_index('No.')
bad_molecules_ts

Original data:  351 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [12]:
y_ts[:] = [x for i,x in enumerate(y_ts) if i not in y_bad_index]

In [13]:
len(y_ts)

351

# 5.Standardization SDF file for test set

In [14]:
all_mols_ts[:] = [x for i,x in enumerate(all_mols_ts) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ts)):
    record = Chem.MolToSmiles(all_mols_ts[i])
    records.append(record)

moldf_ts = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ts.append(m)
    
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  351 molecules


In [15]:
records_ts = []
for i in range(len(moldf_ts)):
    record = Chem.MolToSmiles(moldf_ts[i])
    records_ts.append(record)

In [16]:
df_ts = pd.DataFrame(records_ts, columns=["Smiles"])
df_ts.to_csv('datasets/molecule_ts.smi', sep=',', index=False, header=False)

In [17]:
from pathlib import Path

In [18]:
path = Path('feature_name_rfecv_KRFPFP.txt')
feature_name_rfecv_MF = path.read_text().splitlines()

# 6.Descriptor calculation for work set

In [17]:
fp

{'AtomPairs2DCount': 'fingerprints_xml\\AtomPairs2DFingerprintCount.xml',
 'AtomPairs2D': 'fingerprints_xml\\AtomPairs2DFingerprinter.xml',
 'EState': 'fingerprints_xml\\EStateFingerprinter.xml',
 'CDKextended': 'fingerprints_xml\\ExtendedFingerprinter.xml',
 'CDK': 'fingerprints_xml\\Fingerprinter.xml',
 'CDKgraphonly': 'fingerprints_xml\\GraphOnlyFingerprinter.xml',
 'KlekotaRothCount': 'fingerprints_xml\\KlekotaRothFingerprintCount.xml',
 'KlekotaRoth': 'fingerprints_xml\\KlekotaRothFingerprinter.xml',
 'MACCS': 'fingerprints_xml\\MACCSFingerprinter.xml',
 'PubChem': 'fingerprints_xml\\PubchemFingerprinter.xml',
 'SubstructureCount': 'fingerprints_xml\\SubstructureFingerprintCount.xml',
 'Substructure': 'fingerprints_xml\\SubstructureFingerprinter.xml'}

### KlekotaRoth FPs

In [18]:
from padelpy import padeldescriptor

fingerprint = 'KlekotaRoth'

fingerprint_output_file = ''.join([fingerprint,'.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='datasets/molecule_ws.smi', 
                d_file=fingerprint_output_file, 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

In [19]:
descriptors_KlekotaRoth = pd.read_csv('KlekotaRoth.csv')

In [20]:
descriptors_KlekotaRoth

,Name,KRFP1,KRFP2,KRFP3,KRFP4,KRFP5,KRFP6,KRFP7,KRFP8,KRFP9,...,KRFP4851,KRFP4852,KRFP4853,KRFP4854,KRFP4855,KRFP4856,KRFP4857,KRFP4858,KRFP4859,KRFP4860
0,AUTOGEN_molecule_ws_1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AUTOGEN_molecule_ws_2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AUTOGEN_molecule_ws_3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AUTOGEN_molecule_ws_4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AUTOGEN_molecule_ws_5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,AUTOGEN_molecule_ws_1396,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1396,AUTOGEN_molecule_ws_1397,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1397,AUTOGEN_molecule_ws_1398,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1398,AUTOGEN_molecule_ws_1399,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
descriptors_ws = descriptors_KlekotaRoth.drop('Name', axis=1)

In [23]:
x_tr=descriptors_ws[feature_name_rfecv_MF]

In [24]:
y_tr = np.array(y_tr, dtype=np.float32)
len(y_tr)

1400

# 7.Descriptor calculation for test set

In [ ]:
from padelpy import padeldescriptor

fingerprint = 'KlekotaRoth'

fingerprint_output_file = ''.join([fingerprint,'_ts.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='datasets/molecule_ts.smi', 
                d_file=fingerprint_output_file, 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=False,
                fingerprints=True)

In [25]:
descriptors_KlekotaRoth_ts = pd.read_csv('KlekotaRoth_ts.csv')

In [27]:
x_ts = descriptors_KlekotaRoth_ts.drop('Name', axis=1)

In [29]:
x_ts=x_ts[feature_name_rfecv_MF]

In [30]:
x_ts

,KRFP1,KRFP2,KRFP14,KRFP16,KRFP20,KRFP92,KRFP126,KRFP297,KRFP298,KRFP302,...,KRFP4294,KRFP4295,KRFP4317,KRFP4479,KRFP4525,KRFP4695,KRFP4752,KRFP4770,KRFP4835,KRFP4843
0,1,0,0,0,1,0,0,1,1,1,...,0,0,0,0,1,0,0,0,1,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,1,1,0,0,0,0,1,1,0,...,0,0,0,0,0,1,0,0,0,0
3,1,0,0,1,1,0,0,1,1,1,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,0,0,1,1,0,0,1,1,0,...,0,0,0,1,0,0,0,0,0,0
347,1,0,0,1,1,0,0,1,1,0,...,0,0,0,1,0,0,0,0,0,0
348,1,0,0,1,1,0,0,1,1,0,...,0,0,0,1,0,0,0,0,0,0
349,1,0,0,1,1,0,1,1,1,0,...,0,0,0,1,0,0,0,0,0,0


# load the models from disk

In [31]:
best_svm = pickle.load(open('Models/Padels/HDAC3_SVM_KRFPFP.pkl', 'rb'))

In [33]:
best_gbr = pickle.load(open('Models/Padels/HDAC3_GBR_KRFPFP_final_FS.pkl', 'rb'))

# Prediction for CV

In [38]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [39]:
y_pred_CV_svm = cross_val_predict(best_svm, x_tr, y_tr, cv=cv)

In [40]:
y_pred_CV_gbr = cross_val_predict(best_gbr, x_tr, y_tr, cv=cv)

In [41]:
y_pred_con=(y_pred_CV_svm+y_pred_CV_gbr)/2

In [42]:
Q2_CV = round(r2_score(y_tr, y_pred_con), 2)
Q2_CV

0.66

In [43]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_con)),2)
RMSE_CV

0.71

# Prediction for test set's molecules

In [44]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [45]:
y_pred_svm = best_svm.predict(x_ts)

In [46]:
y_pred_gbr = best_gbr.predict(x_ts)

In [47]:
y_pred_con_ts=(y_pred_svm+y_pred_gbr)/2

In [48]:
Q2_TS = round(r2_score(y_ts, y_pred_con_ts), 2)
Q2_TS

0.69

In [49]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_con_ts)), 2)
RMSE_TS

0.69

# Estimating applicability domain. Method - Euclidian distances, K=1

In [50]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [51]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1.414214,2.645751,2.000000,0.000000,1.414214,1.414214,1.414214,5.000000,2.645751,3.464102,...,2.645751,2.000000,1.000000,1.000000,2.000000,2.236068,1.000000,1.000000,2.828427,1.000000
2,1.732051,3.464102,3.872983,0.000000,3.741657,1.732051,2.236068,5.099020,2.645751,3.464102,...,3.605551,2.000000,1.000000,1.000000,2.236068,2.449490,1.000000,1.000000,2.828427,1.414214
3,1.732051,3.464102,4.242641,1.000000,4.358899,2.236068,2.645751,5.099020,2.645751,3.464102,...,3.605551,2.236068,1.000000,1.000000,2.449490,2.449490,1.414214,1.414214,3.162278,1.414214
4,1.732051,3.605551,4.358899,1.732051,4.582576,2.236068,2.645751,5.196152,2.828427,4.000000,...,3.741657,2.449490,1.000000,1.000000,2.449490,2.645751,1.414214,1.414214,3.316625,1.414214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,8.944272,9.110434,8.774964,8.660254,8.602325,9.486833,8.660254,8.774964,9.899495,9.539392,...,9.539392,9.327379,9.433981,9.380832,9.591663,9.486833,9.380832,9.327379,8.717798,9.273618
1396,9.000000,9.110434,8.774964,8.717798,8.660254,9.486833,8.660254,8.831761,9.949874,9.539392,...,9.539392,9.327379,9.433981,9.380832,9.591663,9.486833,9.433981,9.327379,8.774964,9.327379
1397,9.055385,9.165151,8.888194,8.717798,8.660254,9.539392,8.660254,8.831761,10.049876,9.539392,...,9.695360,9.327379,9.433981,9.433981,9.591663,9.486833,9.433981,9.380832,8.831761,9.327379
1398,9.055385,9.165151,9.055385,8.774964,8.660254,9.539392,8.717798,8.888194,10.049876,9.539392,...,9.695360,9.380832,9.433981,9.433981,9.591663,9.539392,9.433981,9.380832,8.888194,9.327379


In [52]:
similarity= neighbors_k

In [53]:
Dmean=np.mean(similarity[1,:])

In [54]:
round(Dmean, 2)

1.74

In [55]:
std=np.std(similarity[1,:])

In [56]:
round(std, 2)

1.2

In [57]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

2.34


In [58]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [59]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,341,342,343,344,345,346,347,348,349,350
0,0.000000,2.449490,4.472136,1.732051,1.732051,2.000000,1.732051,2.236068,2.828427,2.000000,...,0.000000,1.414214,1.000000,0.000000,2.828427,1.000000,1.000000,1.732051,2.449490,2.645751
1,2.645751,3.464102,4.690416,3.316625,2.645751,2.645751,2.645751,2.449490,3.316625,2.645751,...,1.000000,2.000000,1.414214,1.414214,3.000000,1.414214,1.000000,1.732051,2.449490,2.828427
2,3.000000,3.605551,4.898979,3.464102,3.000000,2.828427,2.828427,2.449490,4.358899,3.162278,...,1.414214,2.236068,1.732051,1.732051,3.000000,1.414214,1.000000,1.732051,2.449490,3.000000
3,3.000000,3.605551,4.898979,3.605551,3.316625,3.605551,2.828427,3.741657,4.690416,3.162278,...,1.414214,2.449490,1.732051,1.732051,3.162278,1.732051,1.414214,2.000000,2.645751,3.000000
4,3.162278,3.741657,5.000000,4.795832,4.000000,3.741657,2.828427,4.123106,4.898979,3.605551,...,1.414214,2.449490,1.732051,1.732051,3.162278,1.732051,1.414214,2.000000,2.645751,3.162278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,9.055385,9.433981,8.660254,9.695360,9.055385,9.110434,9.949874,9.110434,9.380832,9.219544,...,9.273618,9.433981,9.273618,9.433981,9.539392,9.327379,9.380832,9.486833,9.591663,9.695360
1396,9.110434,9.486833,8.660254,9.797959,9.055385,9.110434,10.000000,9.165151,9.380832,9.219544,...,9.327379,9.433981,9.273618,9.433981,9.591663,9.327379,9.380832,9.486833,9.591663,9.695360
1397,9.110434,9.591663,8.717798,9.848858,9.055385,9.165151,10.099505,9.165151,9.433981,9.380832,...,9.433981,9.433981,9.380832,9.433981,9.591663,9.327379,9.380832,9.486833,9.695360,9.695360
1398,9.110434,9.591663,8.774964,9.848858,9.055385,9.165151,10.099505,9.219544,9.433981,9.380832,...,9.433981,9.486833,9.433981,9.433981,9.591663,9.327379,9.380832,9.539392,9.695360,9.797959


In [60]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[0.    2.449 4.472 1.732 1.732 2.    1.732 2.236 2.828 2.    3.873 2.
 1.    2.    2.    1.414 1.    2.449 3.873 1.414 1.414 2.646 4.69  1.
 1.732 1.732 2.646 2.236 1.414 1.    4.69  0.    3.    1.414 1.    1.414
 1.732 1.414 1.414 1.414 1.414 5.292 2.828 1.    1.414 1.    3.742 2.646
 1.732 1.732 2.    3.    2.    1.414 1.732 3.742 1.414 1.414 1.    2.
 0.    3.317 1.732 3.606 2.828 1.414 3.873 0.    2.    4.    3.317 1.732
 0.    1.    1.414 1.732 1.414 2.236 2.828 2.449 0.    1.    2.    1.414
 1.    1.    0.    1.732 0.    5.    2.236 5.    1.414 2.    1.732 2.828
 1.414 2.449 0.    1.732 5.196 1.    0.    1.    2.    1.414 4.243 1.414
 1.732 1.    1.    2.236 1.732 1.    1.732 2.449 1.732 2.    2.236 0.
 2.646 3.    2.449 2.    2.449 2.646 1.732 1.414 1.    3.464 2.236 2.828
 1.    2.646 1.    1.732 1.414 1.732 2.236 2.    3.873 1.    2.    1.
 1.414 2.236 1.414 1.414 0.    3.742 1.732 1.    1.732 1.414 2.449 2.646
 1.414 1.732 1.    1.414 1.    0.    1.732 1.414 1.732 0.    1.   

In [61]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True False False  True  True  True  True  True False  True False  True
  True  True  True  True  True False False  True  True False False  True
  True  True False  True  True  True False  True False  True  True  True
  True  True  True  True  True False False  True  True  True False False
  True  True  True False  True  True  True False  True  True  True  True
  True False  True False False  True False  True  True False False  True
  True  True  True  True  True  True False False  True  True  True  True
  True  True  True  True  True False  True False  True  True  True False
  True False  True  True False  True  True  True  True  True False  True
  True  True  True  True  True  True  True False  True  True  True  True
 False False False  True False False  True  True  True False  True False
  True False  True  True  True  True  True  True False  True  True  True
  True  True  True  True  True False  True  True  True  True False False
  True  True  True  True  True  True  True  True  T

In [62]:
print("Coverage = ", sum(cpd_AD) / len(cpd_AD))

Coverage =  0.7578347578347578


In [63]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  0   3   4   5   6   7   9  11  12  13  14  15  16  19  20  23  24  25
  27  28  29  31  33  34  35  36  37  38  39  40  43  44  45  48  49  50
  52  53  54  56  57  58  59  60  62  65  67  68  71  72  73  74  75  76
  77  80  81  82  83  84  85  86  87  88  90  92  93  94  96  98  99 101
 102 103 104 105 107 108 109 110 111 112 113 114 116 117 118 119 123 126
 127 128 130 132 134 135 136 137 138 139 141 142 143 144 145 146 147 148
 150 151 152 153 156 157 158 159 160 161 162 163 164 165 166 167 168 169
 170 171 172 173 174 175 178 179 180 183 184 185 186 187 188 189 190 191
 192 193 194 195 197 198 199 201 202 203 204 206 210 211 212 213 214 215
 216 217 218 220 221 222 223 224 225 226 227 228 230 232 233 234 235 237
 238 239 240 242 244 246 247 248 249 250 251 252 254 255 256 257 258 259
 260 261 263 264 265 266 267 268 269 270 274 275 277 280 281 282 283 284
 286 287 289 291 292 295 296 297 298 299 301 303 304 305 306 307 308 310
 311 312 31

In [64]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [65]:
y_pred_con_ad=list(y_pred_con_ts)

In [66]:
y_pred_con_ad[:] = [x for i,x in enumerate(y_pred_con_ad) if i not in out_Ad]

In [67]:
len(y_pred_con_ad)

266

In [68]:
y_ts_ad=list(y_ts)

In [69]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [70]:
len(y_ts_ad)

266

In [71]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_con_ad), 2)
Q2_TS

0.74

In [72]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_con_ad)), 2)
RMSE_TS

0.62